For this to work we need to have Phantom installed (https://stackoverflow.com/questions/8049520/web-scraping-javascript-page-with-python) in order to be able to render javascript content

In [1]:
from bs4 import BeautifulSoup, SoupStrainer
import urllib
import re
import pandas as pd
from collections import defaultdict

In [2]:
def read_adress_data(pickle_path='data/adresses/copenhagen_adresses.pickle', raw_path='http://dawa.aws.dk/adresser?format=csv&kommunekode=0101'):
    try: 
        print('Loading from pickle')
        df_adress = pd.read_pickle(pickle_path)
    except:
        print(f'Loading from pickle failed. Loading from source ({raw_path}) instead')
        address_path = raw_path     #address_path = 'http://dawa.aws.dk/adresser?format=json&kommunekode=0101'
        with urllib.request.urlopen(address_path) as f:
            data_binary = f.readlines() #.decode()
        data = [re.sub(r'(?!(([^"]*"){2})*[^"]*$),', '', x.decode().strip()).split(',') for x in data_binary] # https://stackoverflow.com/questions/38336518/remove-all-commas-between-quotes
        df_adress = pd.DataFrame(data[1:], columns=data[0])
        df.to_pickle(pickle_path)
    return df_adress

In [3]:
df_adress = read_adress_data()

Loading from pickle


In [4]:
bolig_liste = defaultdict(str)

for page in range(1,2):
    url_in_scope = f'https://www.boliga.dk/resultat?sort=zipCode-a&municipality=101&page={page}'
    print('Scraping', url_in_scope)
    try:
        page = urllib.request.urlopen(url_in_scope)
        #soup = BeautifulSoup(page, 'html.parser')
        #print(soup)
 
        for link in BeautifulSoup(page, 'html.parser', parse_only=SoupStrainer('a')):

            if link.has_attr('href'):
                #print(link['href'])
                if re.match('/bolig/\d*/.*', link['href']):
                    bolig_liste[link['href']] = defaultdict(str)

                
    except Exception as e:
            print("An error occured.", e)
    

Scraping https://www.boliga.dk/resultat?sort=zipCode-a&municipality=101&page=1


In [5]:
#bolig_liste

In [6]:
for i, bolig in enumerate(bolig_liste.keys()):
    if i == 0:
        bolig_url = f'https://www.boliga.dk{bolig}'
        print(bolig_url)
        
        try:
            page = urllib.request.urlopen(url_in_scope)
            soup = BeautifulSoup(page, 'html.parser')
            #print(soup)
            #body = soup.find('body')
            #body_no_tags = body.findChildren(recursive=False)
            #print(body_no_tags)
            tags = {tag.name: 1 for tag in soup.find_all()}
#            [str(tag) for tag in soup.find_all()]
        except Exception as e:
            print("An error occured.", e)

https://www.boliga.dk/bolig/1616756/tyboroen_alle_91__st_2720_vanloese


In [7]:
[str(tag_v) for tag_v in soup.find('html')][2]

look_for_list = ['Marievej', '4.940', '4.945.000', '33.291', 'mar. 2020', 'Boligen blev sat til salg', 
                 '120', '2500 Valby', '2500 Valby','7 dage på markedet', 'Se bolig hos mægler', 'danbolig']

for t in list(tags):
    #print('tag', t)
    tag_values = [str(tag_v) for tag_v in soup.find(t)]
    for j, tag_v in enumerate(tag_values):
        #print(j)
        #if j < 2:
        for lookout in look_for_list:
            if lookout in tag_v:
                print(' '*5, 'Interesting, found', lookout, 'in', t)
#         if tag_v in
#             print(' '*5, j, tag_v)
    

      Interesting, found Marievej in html
      Interesting, found 33.291 in html
      Interesting, found mar. 2020 in html
      Interesting, found Boligen blev sat til salg in html
      Interesting, found 120 in html
      Interesting, found 2500 Valby in html
      Interesting, found 2500 Valby in html
      Interesting, found Se bolig hos mægler in html
      Interesting, found Marievej in body
      Interesting, found 33.291 in body
      Interesting, found mar. 2020 in body
      Interesting, found 120 in body
      Interesting, found 2500 Valby in body
      Interesting, found 2500 Valby in body
      Interesting, found Marievej in body
      Interesting, found Boligen blev sat til salg in body
      Interesting, found 120 in body
      Interesting, found Se bolig hos mægler in body
      Interesting, found Marievej in app-root
      Interesting, found 33.291 in app-root
      Interesting, found mar. 2020 in app-root
      Interesting, found 120 in app-root
      Interesting, 

Some information seems to be missing. Maybe it's because some javascript is not yet calculated.

In [10]:
bolig_url = 'https://www.boliga.dk/bolig/1644897/ingrid_marievej_66__12_4_2500_valby'

In [11]:
# Try 2 - 
# https://stackoverflow.com/questions/29404856/how-can-i-render-javascript-html-to-html-in-python 
# https://stackoverflow.com/questions/29858752/error-message-chromedriver-executable-needs-to-be-available-in-the-path

from selenium import webdriver
from urllib.request import urlopen
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager


url = bolig_url# 'http://www.google.com'
file_name = 'C:/working/ngs-lib/Boligsalg/data/url_data/data.txt'

conn = urlopen(url)
data = conn.read()
conn.close()

file = open(file_name,'wb')
file.write(data)
file.close()

browser = webdriver.Chrome(ChromeDriverManager().install())
#browser = webdriver.Firefox(executable_path=GeckoDriverManager().install())

#browser = webdriver.Chrome()
browser.get('file:///'+file_name)
#browser.get(url)
html = browser.page_source
browser.quit()


Looking for [chromedriver 80.0.3987.106 win32] driver in cache 
File found in cache by path [C:\Users\theone\.wdm\drivers\chromedriver\80.0.3987.106\win32\chromedriver.exe]


In [12]:
look_for_list = ['Marievej',
                     '4.672',
                     '4.945.000',
                     '250.000',
                     '43.377',
                     'mar. 2020',
                     'Boligen blev sat til salg',
                     'A20',
                     '114',
                     '2500 Valby',
                     '6 dage på markedet',
                     'Se bolig hos mægler',
                     'danbolig']

In [17]:
type(html)
soup = BeautifulSoup(html, "html.parser")
tags = {tag.name: 1 for tag in soup.find_all()}

In [20]:
for t in list(tags):
    print('tag', t)
    tag_values = [str(tag_v) for tag_v in soup.find(t)]
    for j, tag_v in enumerate(tag_values):
        print(j, l)
        #if j < 2:
        for lookout in look_for_list:
            if lookout in tag_v:
                print(' '*5, 'Interesting, found', lookout, 'in', t)

tag html
0
1
      Interesting, found Marievej in html
      Interesting, found 4.672 in html
      Interesting, found 4.945.000 in html
      Interesting, found 250.000 in html
      Interesting, found 43.377 in html
      Interesting, found mar. 2020 in html
      Interesting, found Boligen blev sat til salg in html
      Interesting, found A20 in html
      Interesting, found 114 in html
      Interesting, found 2500 Valby in html
      Interesting, found 6 dage på markedet in html
      Interesting, found Se bolig hos mægler in html
      Interesting, found danbolig in html
tag head
tag body
0
      Interesting, found Marievej in body
      Interesting, found 4.672 in body
      Interesting, found 4.945.000 in body
      Interesting, found 250.000 in body
      Interesting, found 43.377 in body
      Interesting, found mar. 2020 in body
      Interesting, found Boligen blev sat til salg in body
      Interesting, found A20 in body
      Interesting, found 114 in body
      Interest

In [52]:
for interest in look_for_list:
    #print(interest)
    if interest in html:
        print(interest, 'is apparant')
    else:
        print(interest, 'not found :()')

#html

Marievej is apparant
4.672 is apparant
4.945.000 is apparant
250.000 is apparant
43.377 is apparant
mar. 2020 is apparant
Boligen blev sat til salg is apparant
A20 is apparant
114 is apparant
2500 Valby is apparant
6 dage på markedet is apparant
Se bolig hos mægler is apparant
danbolig is apparant


In [ ]:
# TRY 1: Fialing.. ..

#soup = BeautifulSoup(page, 'html.parser')
#print(soup)



#https://stackoverflow.com/questions/8049520/web-scraping-javascript-page-with-python
from requests_html import HTMLSession, AsyncHTMLSession

# session = HTMLSession()
# r = session.get(bolig_url)
# r.html.render()


import asyncio
if asyncio.get_event_loop().is_running(): # Only patch if needed (i.e. running in Notebook, Spyder, etc)
    import nest_asyncio
    nest_asyncio.apply()


session = AsyncHTMLSession()
r = await session.get(bolig_url)
#await r.html.arender()
#resp=r.html.raw_html
#print(resp)